In [106]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [107]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [108]:
# url for https://climate.weather.gc.ca/ historical weather API
url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month=1&timeframe=2&submit=Download+Data"

In [109]:
def download_weather_data(station, year):
    url = url_template.format(station=station, year=year)
    weather_data = pd.read_csv(url, index_col='Date/Time', parse_dates=True)
    # replace the degree symbol in the column names, then change column names to lower case
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
    weather_data.columns = [col.lower() for col in weather_data.columns]
    weather_data = weather_data[['min temp (c)', 'max temp (c)']]
    return weather_data

In [110]:
def query_weather_API(id = 50430):
    df_list = []
    today = date.today()
    for yr in range(today.year-1, today.year+1):
        df = download_weather_data(id, yr)
        df_list.append(df)
    weather = pd.concat(df_list)
    weather.sort_index(inplace=True)
    return weather

In [125]:
end = date.today() - timedelta(days=1)
start = end - timedelta(days=(7*12)-1)
current_weather = query_weather_API()[str(start):str(end)]
current_weather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 84 entries, 2020-07-31 to 2020-10-22
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   min temp (c)  82 non-null     float64
 1   max temp (c)  82 non-null     float64
dtypes: float64(2)
memory usage: 2.0 KB


In [126]:
months = current_weather.index.month.astype(str)
months = months.map(lambda x: x if len(x) == 2 else '0' + x)
days = current_weather.index.day.astype(str)
days = days.map(lambda x: x if len(x) == 2 else '0' + x)
current_weather['month-day'] = months + "-" + days

In [127]:
current_weather

,min temp (c),max temp (c),month-day
Date/Time,,,
2020-07-31,17.5,27.0,07-31
2020-08-01,15.5,28.1,08-01
2020-08-02,14.1,28.2,08-02
2020-08-03,15.1,28.6,08-03
2020-08-04,10.7,25.6,08-04
...,...,...,...
2020-10-18,-8.8,-6.3,10-18
2020-10-19,-8.0,-0.5,10-19
2020-10-20,-4.6,1.9,10-20
